\-- Tracks total revenue per day and computes a cumulative running total to show overall sales growth over time.

\-- It groups orders by CAST(OrderDate AS date), sums TotalDue, and uses a window SUM(...) OVER(ORDER BY OrderDate) for the cumulative total.

\-- This query is special because it turns raw transactions into a simple time-series view that instantly shows business momentum and trend.

In [ ]:
SELECT
    CAST(OrderDate AS date) AS OrderDate,
    SUM(TotalDue)           AS DailySales,
    SUM(SUM(TotalDue)) OVER
        (ORDER BY CAST(OrderDate AS date)) AS RunningSales
FROM Sales.SalesOrderHeader
GROUP BY CAST(OrderDate AS date)

\-- Finds the most recent order placed by each customer so the business can identify last contact points or active customers.

\-- It uses ROW\_NUMBER() OVER (PARTITION BY CustomerID ORDER BY OrderDate DESC) inside a subquery and filters on rn = 1.

\-- What makes it special is that it cleanly solves a “top 1 per group” problem without self-joins or complex subqueries.

In [ ]:
SELECT * FROM (
    SELECT
        CustomerID,
        SalesOrderID,
        OrderDate,
        ROW_NUMBER() OVER
            (PARTITION BY CustomerID ORDER BY OrderDate DESC) AS rn
    FROM Sales.SalesOrderHeader
) AS d
WHERE rn = 1

\-- Ranks all products by total revenue so managers can see which items are best sellers

\-- It groups by ProductID, sums LineTotal, then applies RANK() OVER(ORDER BY SUM(LineTotal) DESC) to order them by performance.

\-- The special part is the ranking window function, which not only sorts products but also handles ties in revenue correctly.

In [ ]:
SELECT
    sod.ProductID,
    SUM(sod.LineTotal) AS TotalSales,
    RANK() OVER (ORDER BY SUM(sod.LineTotal) DESC) AS SalesRank
FROM Sales.SalesOrderDetail AS sod
GROUP BY sod.ProductID

\-- Shows daily sales along with the previous day’s sales and the difference, highlighting spikes or drops in revenue.

\-- A CTE first aggregates daily totals, then LAG(DailySales) OVER(ORDER BY OrderDate) pulls the prior day for each row and a simple subtraction gives the delta.

\-- It’s special because it turns static totals into a comparative trend, making anomalies and seasonality immediately visible.

In [ ]:
WITH Daily AS (
    SELECT
        CAST(OrderDate AS date) AS OrderDate,
        SUM(TotalDue)           AS DailySales
    FROM Sales.SalesOrderHeader
    GROUP BY CAST(OrderDate AS date)
)
SELECT
    OrderDate,
    DailySales,
    LAG(DailySales) OVER (ORDER BY OrderDate)            AS PrevDaySales,
    DailySales - LAG(DailySales) OVER (ORDER BY OrderDate) AS Delta
FROM Daily

\-- Lists every order while also showing the total number of orders stored in the system.

\-- It selects from SalesOrderHeader and uses COUNT(\*) OVER() (no partition) to compute one global count shared across all rows.

\-- This is special because it demonstrates how a window aggregate can attach global context (like “size of dataset”) to each individual row without a separate query.

In [ ]:
SELECT
    SalesOrderID,
    CustomerID,
    OrderDate,
    COUNT(*) OVER () AS TotalOrdersInSystem
FROM Sales.SalesOrderHeader

\--Summarizes revenue per year and includes an extra row with the overall grand total for all years combined.

\--It groups by ROLLUP (YEAR(OrderDate)), which automatically adds the “all years” grouping on top of the yearly groups.

\--This query is special because a single GROUP BY construct returns both detailed and summary views, perfect for reports and dashboards.

In [ ]:
SELECT
    YEAR(OrderDate) AS SalesYear,
    SUM(TotalDue)   AS YearSales
FROM Sales.SalesOrderHeader
GROUP BY ROLLUP (YEAR(OrderDate))

\--Provides sales totals by territory and an overall company total in the same result set.

\--It uses GROUP BY GROUPING SETS ((TerritoryID), ()) to explicitly request both per-territory and grand-total aggregations.

\--What makes it special is the flexibility: you can design exactly which subtotal combinations you want, without multiple UNIONed queries.

In [ ]:
SELECT
    TerritoryID,
    SUM(TotalDue) AS TotalSales
FROM Sales.SalesOrderHeader
GROUP BY GROUPING SETS
(
    (TerritoryID), -- per territory
    ()             -- grand total
)

\-- Segments customers into three spend tiers (high, medium, low) based on their lifetime revenue.

\-- It first aggregates SUM(TotalDue) per CustomerID, then applies NTILE(3) OVER(ORDER BY LifetimeValue DESC) to assign each customer to a tier.

\-- The special aspect is the use of NTILE for simple, data-driven customer segmentation that can feed marketing or loyalty programs.

In [ ]:
WITH CustTotals AS (
    SELECT CustomerID, SUM(TotalDue) AS LifetimeValue
    FROM Sales.SalesOrderHeader
    GROUP BY CustomerID
)
SELECT
    CustomerID,
    LifetimeValue,
    NTILE(3) OVER (ORDER BY LifetimeValue DESC) AS ValueTier
FROM CustTotals

\-- Transforms product inventory metrics into a unified “metric/value” list so analysts can compare safety stock and reorder points side by side.

\-- It selects from Production.Product and uses UNPIVOT (MetricValue FOR MetricName IN (SafetyStockLevel, ReorderPoint)) to convert columns into rows.

\-- This query is special because it normalizes denormalized columns, making it easier to run generic analytics or visualizations across multiple metrics.

In [ ]:
SELECT
    ProductID,
    Name AS ProductName,
    MetricName,
    MetricValue
FROM Production.Product
UNPIVOT (
    MetricValue FOR MetricName IN (SafetyStockLevel, ReorderPoint)
) 
    AS u

\--Identifies orders that are unusually large relative to each customer’s typical spending level.

\--It calculates each customer’s average order value using AVG(TotalDue) OVER (PARTITION BY CustomerID) inside a CTE, then filters for orders where the actual TotalDue is more than 150% of that average.

\--This query is special because it uses a window function to benchmark each order against that customer’s personal history, making it ideal for spotting big-ticket purchases, anomalies, or VIP-level transactions without needing extra joins or subqueries.

In [ ]:
WITH CustomerOrderStats AS (
    SELECT
        SalesOrderID,
        CustomerID,
        OrderDate,
        TotalDue,
        AVG(TotalDue) OVER
            (PARTITION BY CustomerID) AS AvgCustomerOrder
    FROM Sales.SalesOrderHeader
)
SELECT
    SalesOrderID,
    CustomerID,
    OrderDate,
    TotalDue,
    AvgCustomerOrder
FROM CustomerOrderStats
WHERE TotalDue > 1.5 * AvgCustomerOrder
ORDER BY CustomerID, OrderDate